In [9]:
%load_ext autoreload
%autoreload 2

In [1]:
from From_Shelter_to_Love.trainer import get_data_2

In [31]:
df_2 = get_data_2()

/home/isa/code/anaiscasilva/from_shelter_to_love/From_Shelter_to_Love/days_in_shelter.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days in Shelter'] = np.ceil((df['DateTime Outcome'] - df['DateTime Intake']) / np.timedelta64(24,'h'))
/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/isa/code/anaiscasilva/from_shelter_to_love/From_Shelter_to_Love/trainer.py:30: SettingWithCopyWarning: 
A value is trying to 

In [3]:
df_2

,Animal ID,Intake Type,Intake Condition,Animal Type,Date of Birth,Outcome Type,Days in Shelter,age_upon_intake_months,age_upon_intake_years,age_upon_outcome_months,age_upon_outcome_years,group_color,neutered_or_spayed,male_or_female
0,A006100,Public Assist,Normal,Dog,2007-07-09,Return to Owner,2.0,72.0,6.0,72.0,6.0,Yellow,1,1.0
1,A047759,Owner Surrender,Normal,Dog,2004-04-02,Transfer,5.0,120.0,10.0,120.0,10.0,Tricolor,1,1.0
2,A134067,Public Assist,Injured,Dog,1997-10-16,Return to Owner,1.0,192.0,16.0,192.0,16.0,Brown,1,1.0
3,A141142,Stray,Aged,Dog,1998-06-01,Return to Owner,1.0,180.0,15.0,180.0,15.0,Black,1,0.0
4,A163459,Stray,Normal,Dog,1999-10-19,Return to Owner,1.0,180.0,15.0,180.0,15.0,Black,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102720,A829991,Stray,Normal,Dog,2019-02-28,Return to Owner,2.0,24.0,2.0,24.0,2.0,Black,0,1.0
102721,A829992,Stray,Normal,Dog,2019-02-28,Return to Owner,2.0,24.0,2.0,24.0,2.0,White,1,0.0
102722,A830034,Stray,Normal,Dog,2019-02-05,Return to Owner,1.0,24.0,2.0,24.0,2.0,Brown,1,1.0
102723,A830080,Owner Surrender,Normal,Dog,2020-03-02,Rto-Adopt,2.0,12.0,1.0,12.0,1.0,Black,1,1.0


In [5]:
df_2.shape

(102725, 14)

In [5]:
df_2.dtypes

Animal ID                          object
Intake Type                        object
Intake Condition                   object
Animal Type                        object
Date of Birth              datetime64[ns]
Outcome Type                       object
Days in Shelter                   float64
age_upon_intake_months            float64
age_upon_intake_years             float64
age_upon_outcome_months           float64
age_upon_outcome_years            float64
group_color                        object
neutered_or_spayed                  int64
male_or_female                    float64
dtype: object

In [25]:
from sklearn.model_selection import train_test_split

y = df_2["Days in Shelter"]
X = df_2.drop(columns = ["Days in Shelter", 'Date of Birth'] , axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 10)

In [26]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer

In [27]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [29]:
preprocessor = ColumnTransformer([
    ('num_transformer', MinMaxScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
    ('cat_transformer', OneHotEncoder(sparse=False, handle_unknown='ignore'), make_column_selector(dtype_include=['object']))])

final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('linear_regression', LinearRegression())])

final_pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  MinMaxScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fcdabf8cee0>),
                                                 ('cat_transformer',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fcdabf8ca60>)])),
                ('linear_regression', LinearRegression())])

In [30]:
from sklearn.model_selection import cross_val_score

# Cross validate pipeline
cross_val_score(final_pipe, X_train, y_train, cv=5, scoring='r2').mean()

/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/joblib/memory.py", line 352, in 

/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/joblib/memory.py", line 352, in 

/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/joblib/memory.py", line 352, in 

nan

In [23]:
from sklearn import set_config ;set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler , FunctionTransformer,OneHotEncoder
from sklearn.compose import make_column_selector,ColumnTransformer

# Impute then Scale for numerical variables
num_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler())])

# Encode categorical varibles 
cat_transformer = OneHotEncoder(handle_unknown='ignore',sparse=False)

# Apply transformations to desired features
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, make_column_selector(dtype_include=['int64',"float64"])),
    ('cat_transformer', cat_transformer, make_column_selector(dtype_include=["object"]))])


final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('linear_regression', LinearRegression())])

In [24]:
from sklearn.model_selection import cross_val_score

# Cross validate pipeline
cross_val_score(final_pipe, X_train, y_train, cv=5, scoring='r2').mean()

/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/joblib/memory.py", line 352, in 

/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/joblib/memory.py", line 352, in 

/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/isa/.pyenv/versions/3.8.6/envs/from_shelter_to_love/lib/python3.8/site-packages/joblib/memory.py", line 352, in 

nan